# Data Filtering and Conversion Notebook

## In this notebook : we are creating a filtered dataset from a much larger database to extract relevant data/columns for further analysis

## Input : 
    - SQLite database path from joining the input meta data and sequence data (in Code cell 3)
    - Query to select columns of interest (in Code cell 3)
    - User-defined filters to analyze a subset of the data (eg. Baseline vs Acute)
            

## Output : 
    - Dataframe object in CSV or SQLite DB (In Code cell 11 and 12)

In [1]:
# load needed libraries
import sqlite3
import os
import pandas as pd

# User input

## Database from Extract

Connect to SQLite database

In [2]:
def connect():
    '''function to return SQLite connection
    '''
    # create connection object representing database
    # point to local db (DB Path)
    return(sqlite3.connect('/media/teamcovid/ForData/fdacovid3.db') )

Create SQLite connection instance

In [3]:
conn = connect()

## Columns to Transform

Generate SQL query

In [4]:
# Query selects needed columns
# Current Example filters by column disease_stage and number of rows
# Current Example creates a function with WHERE clause
def query_str():
    
 
    
    ''' function takes input: None
                       output: parametrized SQL query string
   
    '''

    query = '''SELECT sequence_id, junction_aa, junction_aa_length,\n
    seqtable.sample_processing_id, metadata.subject_id, metadata.study_id, metadata.sex, \n
    metadata.disease_diagnosis, metadata.disease_stage, metadata.intervention \n
    FROM seqtable \n
    INNER JOIN metadata on metadata.sample_processing_id = seqtable.sample_processing_id \n
    WHERE disease_stage = (?)  limit (?)'''
    
    return(query)

Generate a query string instance

In [5]:
query = query_str()

## Constraints in the Query

Generate dataframe from query above

In [6]:
# This creates a pandas dataframe from the database
# You will need to make a  place holder for column name and number of records
def getdata( query, disease_stage,  num_rows ):
    ''' function takes input: 
                            query : SQL query,
                            and two binding parameters in query string: disease_stage, num_rows   
    '''
    
    df = pd.read_sql_query(query_str(),
                           connect(),
                           params = (disease_stage, num_rows),)
    return(df)

 ### Enter your query parameters below

In [7]:
disease_stage1 = "Baseline"

In [8]:
disease_stage2 = "Acute"

In [9]:
num_rows = 1000000

Generate dataframe instance

In [10]:
df1 = getdata(query, disease_stage1, num_rows )
df2 = getdata(query, disease_stage2, num_rows )

# export queried data frame to a CSV file

In [11]:
#df.to_csv("df_for_analysis2.csv")
#df.to_csv("df_for_analysis2.csv")

# export queried data frame to a SQLite db

In [12]:
# export queried data frame to a SQLite db
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

disk_engine = create_engine('sqlite:///disease_stage.db')
Base = declarative_base()

df1.to_sql('Baseline', disk_engine)
df2.to_sql('Acute', disk_engine)


Base.metadata.create_all(disk_engine)

In [14]:
conn.close()

# Load the queried data

In [17]:
df1.iloc[1]

sequence_id                               5f492218061ded731bf08937
junction_aa                                                       
junction_aa_length                                               0
sample_processing_id                      5f491afe6d45ecc67f6d3412
subject_id                                                       2
study_id                ImmuneCODE-COVID-Release-002: COVID-19-ISB
sex                                                           male
disease_diagnosis                                         COVID-19
disease_stage                                             Baseline
intervention                                                      
Name: 1, dtype: object